### **Retropropagación**

El algoritmo de **retropropagación** (backpropagation) es la pieza central del aprendizaje de redes neuronales feed-forward. Permite ajustar los pesos de la red para que la salida se aproxime a un valor deseado, minimizando una función de costo. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(NeuralNetwork, self).__init__()
        self.hidden_layer = nn.Linear(num_inputs, num_hidden)
        self.output_layer = nn.Linear(num_hidden, num_outputs)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        hidden = self.sigmoid(self.hidden_layer(x))
        output = self.sigmoid(self.output_layer(hidden))
        return output

# Cambiando el nombre de la instancia para evitar confusión
modelo = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=2)

# Optimizador y función de pérdida
optimizer = optim.SGD(modelo.parameters(), lr=0.5)
criterion = nn.MSELoss()  # Instancia correctamente MSELoss

# Datos de entrenamiento
inputs = torch.tensor([[0.05, 0.1]], dtype=torch.float32)
targets = torch.tensor([[0.01, 0.99]], dtype=torch.float32)

# Bucle de entrenamiento
for i in range(10000):
    modelo.zero_grad()           # Resetea los gradientes
    outputs = modelo(inputs)     # Pase hacia adelante
    loss = criterion(outputs, targets)  # Calcula la pérdida
    loss.backward()             # Pase hacia atrás
    optimizer.step()            # Actualiza los parámetros

    if i % 1000 == 0:
        print(f'Epoca {i}, Loss: {loss.item()}')

# Código opcionalmente añadido para validación/pruebas
# Supongamos que tenemos datos de validación:
validation_inputs = torch.tensor([[0.1, 0.2]], dtype=torch.float32)
validation_targets = torch.tensor([[0.05, 0.95]], dtype=torch.float32)

# Evaluar el modelo en modo de evaluación
modelo.eval()
with torch.no_grad():  # Desactiva el cálculo de gradientes
    validation_outputs = modelo(validation_inputs)
    validation_loss = criterion(validation_outputs, validation_targets)
    print(f'Pérdida de validación: {validation_loss.item()}')


En el ejemplo:

```python
modelo = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=2)
optimizer = optim.SGD(modelo.parameters(), lr=0.5)
criterion = nn.MSELoss()
```

- Definimos una red muy sencilla (2->2->2), útil para ilustrar paso a paso el flujo de gradientes.
- Usamos **SGD** (descenso por gradiente estocástico) con tasa de aprendizaje 0.5, y **MSELoss** como función de costo, ideal para regresión o salidas continuas en rango (0,1) tras aplicar sigmoide.

Escribimos la arquitectura y el mapeo al código:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        self.hidden_layer = nn.Linear(num_inputs, num_hidden)
        self.output_layer = nn.Linear(num_hidden, num_outputs)
        self.sigmoid = nn.Sigmoid()
```

- **`nn.Linear`** encapsula los pesos $\mathbf{W}$ y sesgos $\mathbf{b}$ de cada capa.
- **`self.sigmoid`** es la función de activación $\sigma(z)=1/(1+e^{-z})$, que introduce no-linealidad y "aplana" valores fuera del rango $[0,1]$.

En la práctica, esta red calcula:

1. $\mathbf{z}^{(1)} = W^{(1)}x + b^{(1)}$  
2. $\mathbf{a}^{(1)} = \sigma(\mathbf{z}^{(1)})$  
3. $\mathbf{z}^{(2)} = W^{(2)}a^{(1)} + b^{(2)}$  
4. $\mathbf{a}^{(2)} = \sigma(\mathbf{z}^{(2)})$  

donde $x$ es el tensor `inputs`.

Dentro del bucle de entrenamiento:

```python
outputs = modelo(inputs)     # Pase hacia adelante
loss = criterion(outputs, targets)
```

- `modelo(inputs)` invoca `forward(x)` que aplica dos veces `self.sigmoid(self.hidden_layer(x))` y luego `self.sigmoid(self.output_layer(hidden))`.
- `criterion(outputs, targets)` calcula  
  $$
    E = \tfrac12\sum_j\bigl(a_j^{(2)} - y_j\bigr)^2,
  $$  
  donde `targets` es el tensor objetivo $\mathbf{y}$.

Este valor escalar `loss` cuantifica cuánto difiere la predicción de la realidad deseada.


Calculamos los gradientes y la retropropagación

```python
loss.backward()             # Pase hacia atrás
```

Al invocar `.backward()`, PyTorch recorre el grafo computacional construido durante el forward, aplicando la **regla de la cadena** para cada parámetro:

1. **Salida → capa de salida**  
   $$
   \delta^{(2)} = (\mathbf{a}^{(2)} - \mathbf{y}) \;\odot\; \sigma'(\mathbf{z}^{(2)})
   $$
2. **Capa de salida → capa oculta**  
   $$
   \delta^{(1)} = \bigl(W^{(2)\,T}\,\delta^{(2)}\bigr)\odot\sigma'(\mathbf{z}^{(1)})
   $$
3. **Gradientes**  
   $\nabla_{W^{(l)}}E = \delta^{(l)}\,(a^{(l-1)})^T,\quad \nabla_{b^{(l)}}E = \delta^{(l)}$.

Internamente, PyTorch acumula estos gradientes en cada tensor de parámetros (`.grad`), listos para que el optimizador los use.


Realizamos la optimización de parámetros:

```python
optimizer.step()            # Actualiza los parámetros
```

- **`optimizer.step()`** toma cada parámetro $ \theta $ y hace  
  $$
    \theta \leftarrow \theta - \eta \,\frac{\partial E}{\partial \theta}
  $$  
  donde $\eta=0.5$.  
- Antes de esto, en cada iteración, se limpia el gradiente:
  ```python
  modelo.zero_grad()
  ```
  para evitar acumular gradientes de pasos anteriores.

Este ciclo (`zero_grad`, `forward`, `loss.backward`, `step`) es la esencia del aprendizaje para la retropropagación.

Mostramos el rol del bucle de entrenamiento y épocas:

```python
for i in range(10000):
    …
    if i % 1000 == 0:
        print(f'Epoca {i}, Loss: {loss.item()}')
```

- Iteramos 10 000 veces sobre el mismo dato (toy example).  
- En redes reales, reemplazaríamos `inputs` y `targets` por **mini-lotes** de datos, mejorando la generalización y la estabilidad del entrenamiento.
- Imprimir la pérdida cada 1 000 iteraciones permite monitorear la **convergencia**: observaremos cómo $E$ decrece gradualmente.

Tras el entrenamiento, pasamos a evaluar:

```python
modelo.eval()             # Desactiva Dropout/BatchNorm (aquí no hay)
with torch.no_grad():    # Desactiva cálculo de gradientes
    validation_outputs = modelo(validation_inputs)
    validation_loss = criterion(validation_outputs, validation_targets)
    print(f'Pérdida de validación: {validation_loss.item()}')
```

- **`modelo.eval()`** pone la red en modo evaluación. Aunque la red no usa **dropout** ni **BatchNorm**, es buena práctica activarlo siempre.
- **`torch.no_grad()`** reduce memoria y acelera la evaluación al omitir el grafo de cálculo de gradientes.
- Comparamos la **pérdida de validación** con la de entrenamiento para detectar **sobreajuste**. Si la validación fuese muy mayor, significaría que la red memorizó el dato de entrenamiento sin generalizar.

**Importancia de los hiperparámetros y posibles extensiones**

1. **Tasa de aprendizaje ($\eta$)**  
   - Un valor demasiado alto ($\eta>1$) puede hacer que el entrenamiento **diverja**.  
   - Uno muy bajo ($\eta<0.01$) ralentiza la convergencia.

2. **Número de iteraciones/épocas**  
   - Ajustar según la complejidad del problema y tamaño de datos reales.
   - Podríamos usar un **scheduler** (`optim.lr_scheduler`) para disminuir $\eta$ durante el entrenamiento.

3. **Función de activación**  
   - Sustituir sigmoide por **ReLU** o **tanh** mejora la **propagación de gradientes** en redes profundas.

4. **Regularización**  
   - Añadir **Dropout** o **weight decay** (L2) reduce sobreajuste en conjuntos de datos más grandes.


#### **Ejercicios**

**Ejercicio 1:** Crea una red neuronal simple (por ejemplo, una red con una capa oculta) desde cero en Python. Implementa manualmente el algoritmo de retropropagación para actualizar los pesos de la red.  Utiliza esta red para entrenar un modelo que pueda clasificar puntos en dos clases basadas en su posición en un plano 2D. Genera datos sintéticos que sean linealmente separables.

**Ejercicio 2:**  Modifica la red del ejercicio 1 para probar diferentes funciones de activación (ReLU, sigmoidal, tanh) en la capa oculta.  Compara el rendimiento del entrenamiento con cada función de activación. Observa cómo cambia la rapidez de convergencia y si alguna configuración es particularmente propensa a problemas como el desvanecimiento o la explosión de gradientes.

**Ejercicio 3:** Añade L2 (ridge) o L1 (lasso) regularización a la red neuronal que creaste en el ejercicio 1. Entrena la red en un conjunto de datos más complejo que introduzca algo de ruido y observa si la regularización ayuda a mejorar la generalización del modelo.

**Ejercicio 4:** Implementa la retropropagación utilizando diferentes tamaños de batch: desde el descenso de gradiente estocástico (un ejemplo a la vez) hasta el descenso de gradiente por lotes (usando todo el conjunto de datos a la vez). Analiza cómo cambian la estabilidad y la velocidad de convergencia del entrenamiento con los diferentes tamaños de batch. Considera incluir una visualización de la disminución de la pérdida a lo largo de las iteraciones para cada configuración de tamaño de batch.

**Ejercicio 5:**  Utiliza la red neuronal del ejercicio 1 y experimenta con diferentes tasas de aprendizaje y números de épocas de entrenamiento. Implementa una búsqueda de cuadrícula o aleatoria para encontrar la combinación óptima de tasa de aprendizaje y número de épocas. Evalúa el rendimiento del modelo en un conjunto de validación para determinar la mejor configuración de hiperparámetros.

In [ ]:
## Tus respuestas

#### **Backpropagation through time (BPTT)**

Intentaremos explicar BPTT, un algoritmo de retropropagación utilizado para entrenar redes neuronales recurrentes (RNN). Mostraremos la derivación matemática así como el código de implementación. Para el código, usaremos PyTorch para poder comparar los gradientes calculados automáticamente por PyTorch con los calculados manualmente. 

En primer lugar, definamos matemáticamente lo que hace una RNN en cada paso de tiempo $t$.

Sea $x_t \in \mathbb{R}^{d_x}$ un vector de entrada en el paso de tiempo $t$. Una RNN de tamaño $d_h$ calcula su estado oculto $h_t$ como:

$$
\begin{align*}
  r_t &= U x_t + V h_{t-1} \\
  h_t &= f \left( r_t \right)
\end{align*}
$$

donde $U \in \mathbb{R}^{d_h \times d_x}$ y $V \in \mathbb{R}^{d_h \times d_h}$ son parámetros, y $f$ es una función de activación. Nota que no incluimos el término de sesgo aquí por simplicidad.

Un vector de salida $y_t \in \mathbb{R}^{d_o}$ entonces puede ser calculado como:

$$
\begin{align*}
  s_t &= W h_t \\
  y_t &= g \left( s_t \right)
\end{align*}
$$

donde $W \in \mathbb{R}^{d_o \times d_h}$ es un parámetro y $g$ es una función de salida no lineal, que depende de la tarea (por ejemplo, softmax para clasificación). Aquí también no incluimos el término de sesgo por simplicidad. Nota que $U$, $V$, y $W$ se comparten a lo largo de los pasos de tiempo, es decir, no hay matrices separadas para cada paso de tiempo.


#### **Proceso**

Sea $L = \mathcal{L}(y_t)$ la pérdida de nuestra RNN. Uno podría intentar derivar los gradientes de $U$, $V$, $W$ de manera ingenua. Por ejemplo, para calcular el gradiente sobre $W$:

$$
\frac{\partial L}{\partial W_{ij}} = \sum_k \frac{\partial L}{\partial (s_t)_k} \frac{\partial (s_t)_k}{\partial W_{ij}}
$$

Dado que

$$
\frac{\partial (s_t)_k}{\partial W_{ij}} =
\begin{cases}
0 & k \neq i \\
(h_t)_j & \text{de lo contrario}
\end{cases}
$$

tenemos

$$
\frac{\partial L}{\partial W_{ij}} = \frac{\partial L}{\partial (s_t)_i} (h_t)_j
$$

O, en notación matricial

$$
\overline{W} = \overline{s}_t h_t^{\top}
$$

donde la notación de barra denota el gradiente de $ L $ con respecto a él, es decir, $\overline{A}_{ij} = \frac{\partial L}{\partial A_{ij}}$ y $\overline{a}_i = \frac{\partial L}{\partial a_i}$.

Sin embargo, esto es **incorrecto**. Dado que $W$ se comparte a lo largo de los pasos de tiempo, contribuye a $s_t$ para *todos* los pasos de tiempo  $t$. Por lo tanto, al calcular el gradiente, necesitamos sumar los gradientes de todos los pasos de tiempo. En otras palabras, lo que necesitamos es:

$$
\overline{W} = \sum_t \overline{s}_t h_t^{\top}
$$

Los gradientes sobre $U$ y $V$ se pueden calcular de manera similar, es decir

$$
\begin{align*}
  \overline{U} &= \sum_t \overline{r}_t x_t^{\top} \\
  \overline{V} &= \sum_t \overline{r}_t h_{t-1}^{\top}
\end{align*}
$$

Ahora, los últimos gradientes. Gradiente sobre $r_t$ es

$$
\begin{align*}
  \frac{\partial L}{\partial (r_t)_i}
  &= \sum_j \frac{\partial L}{\partial (h_t)_j} \frac{\partial (h_t)_j}{\partial (r_t)_i} \\
  &= \frac{\partial L}{\partial (h_t)_i} f' (r_t)_i
\end{align*}
$$

O, en notación vectorial

$$
\overline{r}_t = \overline{h}_t * f' (r_t)
$$

De manera similar, el gradiente sobre $s_t$ se puede mostrar como $\overline{s}_t = \overline{y}_t * g' (s_t)$.

Por último, necesitamos calcular $\overline{h}_t$. Note que $h_t$ contribuye a $s_t$, y si $t$ no es el último paso de tiempo, entonces también contribuye a $r_{t+1}$. Por lo tanto, si $T$ denota el último paso de tiempo, entonces

$$
\begin{align*}
  \frac{\partial L}{\partial (h_T)_i}
  &= \sum_j \frac{\partial L}{\partial (s_T)_j} \frac{\partial (s_T)_j}{\partial (h_T)_i} \\
  &= \sum_j \frac{\partial L}{\partial (s_T)_j} W_{ji}
\end{align*}
$$

y para $t < T$

$$
\begin{align*}
  \frac{\partial L}{\partial (h_t)_i}
  &= \sum_j \frac{\partial L}{\partial (s_t)_j} \frac{\partial (s_t)_j}{\partial (h_t)_i}
  + \sum_k \frac{\partial L}{\partial (r_{t+1})_k} \frac{\partial (r_{t+1})_k}{\partial (h_t)_i} \\
  &= \sum_j \frac{\partial L}{\partial (s_t)_j} W_{ji}
  + \sum_k \frac{\partial L}{\partial (r_{t+1})_k} V_{ki}
\end{align*}
$$

En notación matricial (y combinando los dos)

$$
\overline{h}_t = W^{\top} \overline{s}_t +
\begin{cases}
0 & t = T \\
V^{\top} \overline{r}_{t+1} & \text{de lo contrario}
\end{cases}
$$


Así, los gradientes completos son

$$
\begin{align*}
  \overline{U} &= \sum_t \overline{r}_t x_t^{\top} \\
  \overline{V} &= \sum_t \overline{r}_t h_{t-1}^{\top} \\
  \overline{W} &= \sum_t \overline{s}_t h_t^{\top} \\
  \overline{r}_t &= \overline{h}_t * f' \left( r_t \right) \\
  \overline{s}_t &= \overline{y}_t * g' \left( s_t \right) \\
  \overline{h}_t &= W^{\top} \overline{s}_t +
    \begin{cases}
    0 & t = T \\
    V^{\top} \overline{r}_{t+1} & \text{de lo contrario}
    \end{cases}
\end{align*}
$$

#### **Ejemplo numérico de BPTT (dimensiones escalares, $T=2$)**

**Definiciones**  
- Dimensiones: $d_x=d_h=d_o=1$.  
- Pasos de tiempo: $t=1,2$.  
- Parámetros iniciales:
  $$
    U = 0.5,\quad V = 0.1,\quad W = 0.8.
  $$
- Activaciones: $f = \tanh$, $g$ identidad.
- Entradas:
  $$
    x_1 = 1.0,\quad x_2 = 2.0.
  $$
- Objetivos:
  $$
    y^{\ast}_1 = 0.0,\quad y^{\ast}_2 = 1.0.
  $$
- Pérdida total (suma de MSE en ambos $t$):
  $$
    L = \tfrac12\bigl(y_1 - y^{\ast}_1\bigr)^2 \;+\; \tfrac12\bigl(y_2 - y^{\ast}_2\bigr)^2.
  $$
- Notación:  
  - $r_t = U x_t + V h_{t-1}$, con $h_0=0$.  
  - $h_t = \tanh(r_t)$.  
  - $s_t = W h_t$.  
  - $y_t = s_t$.  


#### **1.  Paso forward**

| Paso | Cálculo                             | Valor                                          |
|:----:|:------------------------------------|:-----------------------------------------------|
| $t=1$ | $r_1 = 0.5\cdot1.0 + 0.1\cdot0 = 0.5$ | $r_1 = 0.5$                                   |
|        | $h_1 = \tanh(0.5)\approx0.4621$     | $h_1\approx0.4621$                            |
|        | $s_1 = 0.8\cdot0.4621\approx0.3697$ | $s_1\approx0.3697$                            |
|        | $y_1 = s_1\approx0.3697$            | $y_1\approx0.3697$                            |
| $t=2$ | $r_2 = 0.5\cdot2.0 + 0.1\cdot0.4621$ | $r_2 = 1.0 + 0.04621 = 1.04621$               |
|        | $h_2 = \tanh(1.04621)\approx0.7809$ | $h_2\approx0.7809$                            |
|        | $s_2 = 0.8\cdot0.7809\approx0.6247$ | $s_2\approx0.6247$                            |
|        | $y_2 = s_2\approx0.6247$            | $y_2\approx0.6247$                            |

La pérdida:
$$
\begin{aligned}
L &= \tfrac12(0.3697 - 0)^2 \;+\; \tfrac12(0.6247 - 1)^2 \\
  &= 0.5\cdot0.1366 + 0.5\cdot0.1408 \approx 0.1387.
\end{aligned}
$$


#### **2. Paso backward: deltas y gradientes en cada $t$**

Denotemos
$\overline{y}_t = \frac{\partial L}{\partial y_t} = y_t - y^{\ast}_t$.

#### Paso $t=2$

1. **Salida → gradiente de $s_2$**  
   $$
   \overline{s}_2
   = \overline{y}_2 \cdot g'(s_2)
   = (0.6247 - 1.0)\cdot1
   = -0.3753.
   $$

2. **Gradiente de $W$** (acumular por todos los $t$)  
   $$
   \frac{\partial L}{\partial W}
   \biggr\rvert_{t=2}
   = \overline{s}_2 \,h_2
   = (-0.3753)\cdot0.7809
   \approx -0.2929.
   $$

3. **Estado oculto → gradiente de $h_2$**  
   $$
   \overline{h}_2
   = W^{\top}\,\overline{s}_2
   = 0.8\cdot(-0.3753)
   = -0.3002
   \quad\text{(no hay término $V^\top \overline r_3$ porque $t=2=T$).}
   $$

4. **Gradiente de $r_2$**  
   $$
   \overline{r}_2
   = \overline{h}_2 \;\ast\; f'(r_2)
   = -0.3002 \;\ast\; \bigl(1 - \tanh^2(1.0462)\bigr)
   = -0.3002\cdot\bigl(1 - 0.7809^2\bigr)
   = -0.3002\cdot0.3904
   \approx -0.1172.
   $$

5. **Gradientes de $U$ y $V$** en $t=2$:  
   $$
   \frac{\partial L}{\partial U}\biggr\rvert_{t=2}
   = \overline{r}_2\,x_2
   = (-0.1172)\cdot2.0
   = -0.2344,
   \qquad
   \frac{\partial L}{\partial V}\biggr\rvert_{t=2}
   = \overline{r}_2\,h_1
   = (-0.1172)\cdot0.4621
   \approx -0.0542.
   $$


##### **Paso $t=1$**

1. **Salida → gradiente de $s_1$**  
   $$
   \overline{s}_1
   = y_1 - y^{\ast}_1
   = 0.3697 - 0
   = 0.3697.
   $$

2. **Gradiente de $W$** (acumular a lo ya obtenido)  
   $$
   \frac{\partial L}{\partial W}
   \biggr\rvert_{t=1}
   = \overline{s}_1\,h_1
   = 0.3697\cdot0.4621
   \approx 0.1709.
   $$
   **Total**  
   $\displaystyle \overline{W}
   = -0.2929 + 0.1709
   = -0.1220.$

3. **Estado oculto → gradiente de $h_1$**  
   Ahora $h_1$ influye en $s_1$ y en $r_2$, así que
   $$
   \overline{h}_1
   = W^{\top}\,\overline{s}_1 \;+\; V^{\top}\,\overline{r}_2
   = 0.8\cdot0.3697 \;+\; 0.1\cdot(-0.1172)
   = 0.2958 - 0.0117
   = 0.2841.
   $$

4. **Gradiente de $r_1$**  
   $$
   \overline{r}_1
   = \overline{h}_1 \,\ast\, (1 - \tanh^2(0.5))
   = 0.2841\cdot\bigl(1 - 0.4621^2\bigr)
   = 0.2841\cdot0.7861
   \approx 0.2233.
   $$

5. **Gradientes de $U$ y $V$** en $t=1$:  
   $$
   \frac{\partial L}{\partial U}\biggr\rvert_{t=1}
   = \overline{r}_1\,x_1
   = 0.2233\cdot1.0
   = 0.2233,
   \qquad
   \frac{\partial L}{\partial V}\biggr\rvert_{t=1}
   = \overline{r}_1\,h_0
   = 0.2233\cdot0
   = 0.
   $$
   **Totales**  
   $$
   \overline{U}
   = -0.2344 + 0.2233
   = -0.0111,
   \qquad
   \overline{V}
   = -0.0542 + 0
   = -0.0542.
   $$


#### 3. Resultados finales de los gradientes

$$
\boxed{
\begin{aligned}
  \overline{U} &\approx -0.0111,  &
  \overline{V} &\approx -0.0542,  &
  \overline{W} &\approx -0.1220.
\end{aligned}
}
$$

Y para cada paso:
$$
\begin{aligned}
  \overline{r}_1 &\approx  0.2233,  & \overline{s}_1 &\approx  0.3697,  & \overline{h}_1 &\approx  0.2841,\\
  \overline{r}_2 &\approx -0.1172,  & \overline{s}_2 &\approx -0.3753,  & \overline{h}_2 &\approx -0.3002.
\end{aligned}
$$

Este procedimiento es la base de BPTT: un forward para guardar $(r_t,h_t,s_t)$, y un backward que recorre $t=T$ hacia $1$ calculando $\overline{h}_t,\overline{r}_t,\overline{s}_t$ y acumulando gradientes.

#### **Anotaciones adicionales**

#### 1. Compartición de parámetros y acumulación de gradientes

En una RNN los mismos parámetros $U,V,W$ se usan en **cada** paso de tiempo. Por tanto:

- Cada aplicación de $W$ en el cálculo de $s_t = W\,h_t$ aporta una "porción" de gradiente $\overline{s}_t\,h_t^\top$.  
- Para obtener el total, **sumamos** esas porciones sobre todos los $t$:
  $$
    \overline{W} \;=\;\sum_{t=1}^T \overline{s}_t\,h_t^\top.
  $$
  
Lo mismo sucede con $U$ y $V$. Esta suma refleja que, al optimizar, cada paso de tiempo "empuja" los pesos en direcciones potencialmente distintas, y la actualización final combina todas esas fuerzas.

#### 2. Flujo de gradiente a través de la activación de salida ($\overline{s}_t$)

Para cada paso $t$, la salida intermedia $s_t = W\,h_t$ genera la predicción $y_t = g(s_t)$. El gradiente

$$
\overline{s}_t \;=\; \frac{\partial L}{\partial y_t}\;g'(s_t)
$$

toma en cuenta:

1. **Error local**: $\frac{\partial L}{\partial y_t}$ mide cuánto contribuye la salida en $t$ al costo total.  
2. **Sensibilidad del no lineal**: $g'(s_t)$ ajusta ese error según la curvatura de la función $g$.  

Ese producto nos dice "cuánto deberíamos cambiar" el vector $s_t$.


#### 3. Transmisión del error hacia el estado oculto ($\overline{h}_t$)

El estado $h_t$ influye **directamente** en la pérdida a través de $s_t$, **y** (si no es el último paso) influye **indirectamente** al afectar el siguiente pre-activación $r_{t+1} = U x_{t+1} + V h_t$. Por eso

$$
\overline{h}_t
= W^\top\,\overline{s}_t 
\;+\;
\begin{cases}
0, & t=T,\\
V^\top\,\overline{r}_{t+1}, & t<T.
\end{cases}
$$

- El primer término, $W^\top\,\overline{s}_t$, "retrae" el error de la capa de salida.  
- El segundo término, $V^\top\,\overline{r}_{t+1}$, trae la contribución de **pasos futuros**. Esto es lo que convierte la retropropagación en "por tiempo" (temporal): el error circula hacia atrás por la conexión recurrente.


#### 4. Paso al pre-activación ($\overline{r}_t$)

Una vez que sabemos $\overline{h}_t$, lo transferimos a la variación en el pre-activación:

$$
\overline{r}_t = \overline{h}_t \;\ast\; f'(r_t),
$$

donde $f'$ es la derivada de la no lineal interna (por ejemplo $\tanh'$ o sigmoid’). Este factor **modula** cuánto del error de $h_t$ se atribuye a cada valor de $r_t$, según la pendiente local de $f$.  

**Importante**: si $f'$ es muy pequeño (como en saturación), hace que $\overline{r}_t$ se atenúe drásticamente—origen del problema de **vanishing gradients** en RNNs largas.


#### 5. Contribución a $U$ y $V$

Con $\overline{r}_t$ definido, podemos asignar qué fracción de la variación total recae sobre $U$ y $V$ en ese paso:

$$
\begin{aligned}
\frac{\partial L}{\partial U} &\biggr\rvert_{t}
= \overline{r}_t \;x_t^\top, 
&
\frac{\partial L}{\partial V} &\biggr\rvert_{t}
= \overline{r}_t \;h_{t-1}^\top.
\end{aligned}
$$

Nuevamente, como estos parámetros se reutilizan, se suman sobre $t=1\dots T$.


#### 6. Interpretación como grafo acíclico "desplegado"

Una forma de visualizarlo es **desplegando** la RNN en el tiempo, convirtiéndola en una red feed-forward muy profunda de $T$ "capas" donde:

- Cada capa comparte los mismos pesos $U,V,W$.  
- El gradiente fluye hacia atrás combinando rutas verticales (salida) y diagonales (recurrente).  

Este grafo deja claro por qué hay que acumular a lo largo de las dos direcciones: espacial (capa a capa) y temporal (paso $t$ a $t-1$).


#### 7. Validación con el ejemplo numérico

En el ejemplo escalar con $T=2$:

1. Se calculó $\overline{s}_2$, $\overline{h}_2$, $\overline{r}_2$, y sus correspondientes contribuciones $\overline{W}$, $\overline{U}$, $\overline{V}$.  
2. Luego, en $t=1$, la "segunda vía" $V^\top\,\overline{r}_2$ entró en $\overline{h}_1$, mostrando cómo el error de $t=2$ retrocede a $t=1$.  
3. Finalmente, al sumar las dos contribuciones de $\overline{s}_1\,h_1$ y $\overline{s}_2\,h_2$, surgió el total de $\overline{W}$.

Cada término numérico coincide exactamente con las fórmulas matriciales vistas arriba, confirmando la **coherencia** de la derivación.

En general:

- El **secreto** de BPTT está en rastrear dos flujos de error: el que va hacia la salida inmediata y el que se propaga hacia atrás en el tiempo.  
- La compartición de pesos exige **sumar** las contribuciones de cada paso.  
- La fórmula de $\overline{h}_t$ es la clave: mezcla el gradiente "local" ($W^\top\overline{s}_t$) con el "futuro" ($V^\top\overline{r}_{t+1}$), y permite capturar dependencia temporal.  


#### **Implementacion en PyTorch**

La implementación del cálculo hacia adelante (forward) es bastante directa. Comenzamos desde el paso de tiempo 1, y calculamos $r_t$, $h_t$, $s_t$ y $y_t$, dados los inputs $x_t$ y $h_{t-1}$, hasta el último paso de tiempo $T$. Comenzamos desde el paso de tiempo 1 porque en la relación de recurrencia, para calcular $r_t$ necesitamos $h_{t-1}$. Por lo tanto, el paso de tiempo anterior debe completarse primero.

La implementación del cálculo hacia atrás es similar, pero en cambio comenzamos desde el último paso de tiempo $T$ hasta el primero. Esto se debe a que en la relación de recurrencia, necesitamos $\overline{r}_{t+1}$ para calcular $\overline{h}_t$. Así, el siguiente paso de tiempo debe completarse de antemano. Además, a medida que iteramos, necesitamos acumular los gradientes en $U$, $V$ y $W$.

Primero, vamos a crear una clase `RNNCell`. Esta clase es responsable de un solo paso de tiempo. 

Tiene los métodos forward y backward que realizan el cálculo hacia adelante y hacia atrás, respectivamente.

In [ ]:
import torch

In [ ]:
class RNNCell:
    """Una celda RNN responsable de un solo paso de tiempo.

    Args:
        inp_sz (int): Tamaño de la entrada.
        hid_sz (int): Tamaño del estado oculto.
        out_sz (int): Tamaño de la salida.
    """
    def __init__(self, inp_sz, hid_sz, out_sz):
        self.inp_sz = inp_sz
        self.hid_sz = hid_sz
        self.out_sz = out_sz

        # U, V, W son los parámetros, por lo que establecemos requires_grad=True
        # para indicar a PyTorch que necesitamos calcular los gradientes
        self.U = torch.empty(hid_sz, inp_sz, requires_grad=True)
        self.V = torch.empty(hid_sz, hid_sz, requires_grad=True)
        self.W = torch.empty(out_sz, hid_sz, requires_grad=True)

        # Estos son los gradientes en U, V y W que calcularemos
        # manualmente. También los compararemos con
        # los gradientes calculados por PyTorch para ver si nuestro
        # cálculo de gradientes es correcto.
        self.U_grad = torch.zeros_like(self.U)
        self.V_grad = torch.zeros_like(self.V)
        self.W_grad = torch.zeros_like(self.W)
        
        self.reset_parameters()

    def reset_parameters(self):
        """Inicializar parámetros.

        Los parámetros se inicializarán desde la distribución uniforme U(-0.1, 0.1).
        """
        s = 0.1  # un valor mayor puede hacer que los gradientes exploten
        torch.nn.init.uniform_(self.U, -s, s)
        torch.nn.init.uniform_(self.V, -s, s)
        torch.nn.init.uniform_(self.W, -s, s)
    def zero_grad(self):
        """Se pone el gradiente a cero"""
        self.U_grad.zero_()
        self.V_grad.zero_()
        self.W_grad.zero_()
    
    def forward(self, x, hp):
        """Realizar el cálculo hacia adelante.
        
        Args:
            x (Tensor): Entrada en el paso de tiempo actual.
            hp (Tensor): Estado oculto en el paso de tiempo anterior.
            
        Returns:
            Tensor: Salida en el paso de tiempo actual.
            Tensor: Estado oculto en el paso de tiempo actual.
        """
        _, h, _, y = self._get_internals(x, hp)
        return y, h

    def backward(self, y_grad, rn_grad, x, hp):
        """Realizar el cálculo hacia atrás.
        
        Args:
            y_grad (Tensor): Gradiente sobre la salida en el paso de tiempo actual.
            rn_grad (Tensor): Gradiente sobre el vector r en el siguiente paso de tiempo.
            x (Tensor): Entrada en el paso de tiempo actual que se pasó a `forward`.
            hp (Tensor): Estado oculto en el paso de tiempo anterior que se pasó a `forward`.
            
        Returns:
            Tensor: Gradiente sobre el vector r en el paso de tiempo actual.
        """
        # Obtener los vectores internos r, h, y s del cálculo hacia adelante
        r, h, s, _ = self._get_internals(x, hp)

        s_grad = y_grad * torch.sigmoid(s) * (1-torch.sigmoid(s))
        h_grad = self.W.t().matmul(s_grad) + self.V.t().matmul(rn_grad)
        r_grad = h_grad * torch.sigmoid(r) * (1-torch.sigmoid(r))

        # Los gradientes de los parámetros se acumulan
        self.U_grad += r_grad.view(-1, 1).matmul(x.view(1, -1))
        self.V_grad += r_grad.view(-1, 1).matmul(hp.view(1, -1))
        self.W_grad += s_grad.view(-1, 1).matmul(h.view(1, -1))

        return r_grad

    def _get_internals(self, x, hp):
        # Estos son los calculos forward reales
        r = self.U.matmul(x) + self.V.matmul(hp)
        h = torch.sigmoid(r)
        s = self.W.matmul(h)
        y = torch.sigmoid(s)
        
        return r, h, s, y


Ahora, creemos una clase RNN. Esta clase acepta una `RNNCell` y es responsable de realizar la iteración sobre los pasos de tiempo.

In [ ]:
class RNN:
    def __init__(self, cell):
        self.cell = cell
    
    def forward(self, xs, h0):
        """Realiza la computación hacia adelante para todos los pasos de tiempo.
        
        Args:
            xs (Tensor): Tensor 2-D de entradas para cada paso de tiempo. La
                primera dimensión corresponde al número de pasos de tiempo.
            h0 (Tensor): Estado oculto inicial.
            
        Returns:
            Tensor: Tensor 2-D de salidas para cada paso de tiempo. La primera
                dimensión corresponde al número de pasos de tiempo.
            Tensor: Tensor 2-D de estados ocultos para cada paso de tiempo más
                `h0`. La primera dimensión corresponde al número de pasos de
                tiempo.
        """
        ys, hs = [], [h0]  # Inicializa listas vacías para almacenar las salidas y los estados ocultos
        for x in xs:
            y, h = self.cell.forward(x, hs[-1])  # Calcula la salida y el siguiente estado oculto
            ys.append(y)  # Agrega la salida a la lista de salidas
            hs.append(h)  # Agrega el estado oculto al final de la lista de estados ocultos
        return torch.stack(ys), torch.stack(hs)  # Apila las salidas y los estados ocultos en tensores
    
    def backward(self, ys_grad, xs, hs):
        """Realiza la computación hacia atrás para todos los pasos de tiempo.
        
        Args:
            ys_grad (Tensor): Tensor 2-D de los gradientes en las salidas
                para cada paso de tiempo. La primera dimensión corresponde a
                el número de pasos de tiempo.
            xs (Tensor): Tensor 2-D de entradas para cada paso de tiempo que
                fue pasado a `forward`.
            hs (Tensor): Tensor 2-D de estados ocultos que es devuelto por
                `forward`.
        """
        # Para el último paso de tiempo, el gradiente en r es cero
        rn_grad = torch.zeros(self.cell.hid_sz)  # Inicializa el gradiente de la celda recurrente como cero

        for y_grad, x, hp in reversed(list(zip(ys_grad, xs, hs))):
            rn_grad = cell.backward(y_grad, rn_grad, x, hp)  # Calcula el gradiente para cada paso de tiempo


A continuación, calculamos la función de pérdida. Aquí utilizamos una pérdida simple de suma de cuadrados.

In [ ]:
def compute_loss(ys, ts):
    return 0.5 * torch.sum((ys - ts)**2)

Ahora, construimos la RNN. Tendrá un tamaño de entrada de 2, un tamaño oculto de 3 y un tamaño de salida de 4.

In [ ]:
cell = RNNCell(2, 4, 5)
rnn = RNN(cell)

Luego, creamos entradas y objetivos ficticios para la red recurrente. Estableceremos el número de pasos de tiempo en 3.

In [ ]:
xs = torch.rand(3, cell.inp_sz)
hp = torch.rand(cell.hid_sz)
ts = torch.rand(3, cell.out_sz)

A continuación, realizamos la computación del forward y calculamos la pérdida.

In [ ]:
ys, hs = rnn.forward(xs, hp)
loss = compute_loss(ys, ts)

In [ ]:
loss


Veamos los gradientes calculados por PyTorch.


In [ ]:
loss.backward()

In [ ]:
rnn.cell.U.grad

In [ ]:
rnn.cell.V.grad

In [ ]:
rnn.cell.W.grad

Ahora, ejecutemos nuestra computación backward y comparemos el resultado con el de PyTorch.

In [ ]:
# Esto se obtiene de nuestra función de pérdida de suma de cuadrados
ys_grad = ys - ts

with torch.no_grad():  # necesario para que PyTorch no genere un erro
    rnn.cell.zero_grad()
    rnn.backward(ys_grad, xs, hs)

Ahora veamos si nuestros gradientes calculados manualmente son correctos, es decir, si son iguales a los de PyTorch.

In [ ]:
rnn.cell.U_grad

In [ ]:
rnn.cell.V_grad

In [ ]:
rnn.cell.W_grad

> Pregunta: comprueba si los resultados son iguales.

### **Ejercicios**

**Ejercicio 1**: Dado un simple RNN con una capa oculta y una función de activación sigmoide, realiza la derivación manual de los gradientes para los parámetros $U$, $V$ y $W$ utilizando BPTT. Asume que la RNN tiene una sola unidad oculta y procesa secuencias de tres tiempos.

**Ejercicio 2**: Implementa una RNN simple en Python sin usar librerías de deep learning como TensorFlow o PyTorch. Tu RNN debe incluir métodos de forward y backward, y debe ser capaz de procesar secuencias de longitud variable.

**Ejercicio 3**: Utiliza PyTorch para crear una RNN simple y compara los gradientes que calcula automáticamente con los que calculaste manualmente en el ejercicio 1 o implementaste en el ejercicio 2.

**Ejercicio 4:** Modifica la RNN implementada en el ejercicio 2 para incluir secuencias de entrada de diferentes longitudes. Experimenta con inicializaciones de parámetros y observa cómo afectan a la magnitud de los gradientes durante el entrenamiento.

**Ejercicio 5:** Implementa varias funciones de activación (ReLU, tanh, sigmoide) en tu RNN del ejercicio 2. Entrena tu modelo en un conjunto de datos de prueba simple (puede ser generado sintéticamente) y compara cómo la elección de la función de activación afecta el rendimiento.



In [ ]:
## Tus respuestas